In [3]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [13]:
%load_ext autoreload
%autoreload
exec(open("plot_utils.py").read())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
country_name = 'Sweden'
file_name = '../output/2020-05-09-deltat0-more-steps-normal/inference/{}.npz'.format(country_name)
data = np.load(file_name)



In [18]:
df = pd.read_csv('../data/NPI_dates.csv',parse_dates=['First','Last'])
df[df['Country']=='United_Kingdom']['First'].iloc[0].to_pydatetime()

IndexError: single positional indexer is out-of-bounds

In [6]:
var_names = list(data['var_names'])
incidences = data['incidences']
chain = data['chain']
nsteps, ndim, N, Td1, Td2, τ_model = data['params']
nburn = int(nsteps * 0.6)
sample = chain[:, nburn:, :].reshape(-1, ndim)

medians = [np.median(sample[:,i]) for i in range(len(var_names))]
means = [sample[:,i].mean() for i in range(len(var_names))]
lnprobability = data['lnprobability'][:, nburn:]
MAP = sample[lnprobability.argmax()]

In [8]:
list(zip(var_names, means, medians))

[('Z', 3.8984625194248097, 4.0560039618509816),
 ('D', 3.494481433233447, 3.487784466814483),
 ('μ', 0.4805096811929219, 0.4292044874748192),
 ('β', 1.085728823946531, 1.055669610679371),
 ('α1', 0.18640106847592025, 0.10201995949103443),
 ('λ', 0.6174071198426647, 0.6174563374801387),
 ('α2', 0.3127706389991906, 0.2430407527564623),
 ('E0', 522.3487911364391, 261.59729603294494),
 ('Iu0', 645.9161956900988, 340.9878457191422),
 ('Δt0', 3.0285386427376513, 2.786489306197325),
 ('τ', 18.831399862722208, 18.1631254331068)]

## medians

In [16]:
seed_max = 3000
params_bounds = {
    'Z' : (2, 5),
    'D' : (2, 5),
    'μ' : (0.2, 1),
    'β' : (0.8, 1.5),
    'α1' : (0.02, 1),
    'λ'  : (0, 1),
    'α2' : (0.02, 1),
    'E0' : (0, seed_max),
    'Iu0' : (0, seed_max),
    'Δt0' : (1,5) #how much zeros before the first incident
}

## loglikelihood code:
θ = medians
X = incidences
Z, D, μ, β, α1, λ, α2, E0, Iu0, Δt0, τ = θ
Δt0 = int(Δt0)
τ = int(τ) # for explanation see https://github.com/dfm/emcee/issues/150

total_zeros = params_bounds['Δt0'][1]
unrellevant_zeros = total_zeros - Δt0
τ = τ - unrellevant_zeros
X = X[unrellevant_zeros:]
ndays = len(X)

S, E, Ir, Iu, R, Y = simulate(Z, D, μ, β, α1, λ, α2, E0, Iu0, Δt0, τ, ndays, N)
p1 = 1/Td1
p2 = 1/Td2
Xsum = X.cumsum() 
n = Y[1:] - Xsum[:-1] 
n = np.maximum(0, n)
p = ([p1] * τ + [p2] * (ndays - τ))[1:]

loglik = scipy.stats.poisson.logpmf(X[Δt0:], n[Δt0-1:] * p[Δt0-1:])

In [15]:
def r(arr):
    return [str(int(a)) if not np.isinf(a) else '-inf' for a in arr]
def r_arrays(arr):
    return [r(a) for a in arr]

print('\t'.join(['day', 'loglik', 'X', 'Xsum', 'Y', 'n']))
arr = r_arrays(list(zip(range(len(X)),loglik,X, Xsum, Y, n)))
for a in arr:
    print('\t'.join(a))

day	loglik	X	Xsum	Y	n
0	-1	0	0	0	7
1	-2	0	0	7	16
2	-1	1	1	16	26
3	-3	5	6	27	33
4	-4	5	11	39	41
5	-5	1	12	52	55
6	-2	1	13	67	70
7	-2	1	14	83	86
8	-8	9	23	100	97
9	-81	11	34	120	107
10	-18	26	60	141	103
11	-46	76	136	163	53
12	-61	24	160	189	56
13	-138	42	202	216	44
14	-inf	45	247	246	32
15	-inf	78	325	279	0
16	-inf	136	461	315	0
17	-inf	158	619	355	0
18	-inf	155	774	355	0
19	-inf	149	923	452	0
20	-inf	108	1031	548	0
21	-inf	89	1120	645	0
22	-inf	46	1166	744	0
23	-inf	134	1300	847	0
24	-inf	122	1422	952	0
25	-inf	200	1622	1061	0
26	-inf	123	1745	1174	0
27	-inf	160	1905	1292	0
28	-inf	110	2015	1413	0
29	-inf	256	2271	1539	0
30	-inf	238	2509	1669	0


## means

In [134]:
## loglikelihood code:
θ = means
X = incidences
Z, D, μ, β, α1, λ, α2, E0, Iu0, Δt0, τ = θ
τ = int(τ) # for explanation see https://github.com/dfm/emcee/issues/150
Δt0 = int(Δt0)

ndays = len(X)
S, E, Ir, Iu, R, Y = simulate(*θ, ndays, N)
p1 = 1/Td1
p2 = 1/Td2
Xsum = X.cumsum() 
n = Y[1:] - Xsum[:-1] 
n = np.maximum(0, n)
p = ([p1] * τ + [p2] * (ndays - τ))[1:]

loglik = scipy.stats.poisson.logpmf(X[Δt0:], n[Δt0-1:] * p[Δt0-1:])

In [135]:
def r(arr):
    return [str(int(a)) if not np.isinf(a) else '-inf' for a in arr]
def r_arrays(arr):
    return [r(a) for a in arr]

print('\t'.join(['day', 'loglik', 'X', 'Xsum', 'Y', 'n']))
arr = r_arrays(list(zip(range(len(X)),loglik,X, Xsum, Y, n)))
for a in arr:
    print('\t'.join(a))

day	loglik	X	Xsum	Y	n
0	-12	0	0	0	29
1	-18	0	0	29	66
2	-21	0	0	66	110
3	-19	0	0	110	162
4	-26	0	0	162	220
5	-45	1	1	220	287
6	-55	5	6	288	359
7	-68	5	11	365	441
8	-59	1	12	452	540
9	-68	1	13	552	652
10	-56	1	14	665	780
11	-19	9	23	794	918
12	-87	11	34	941	1075
13	-78	26	60	1109	1239
14	-94	76	136	1299	1381
15	-119	24	160	1517	1604
16	-101	42	202	1764	1843
17	-119	45	247	2045	1798
18	-163	78	325	2045	2222
19	-218	136	461	2547	2596
20	-338	158	619	3057	2968
21	-448	155	774	3587	3370
22	-640	149	923	4144	3808
23	-535	108	1031	4731	4322
24	-648	89	1120	5353	4894
25	-594	46	1166	6014	5549
26	-840	134	1300	6715	6160
27	-872	122	1422	7460	6829
28	-1110	200	1622	8251	7470
29	-924	123	1745	9092	8241
30	-1076	160	1905	9986	9030
31	-1091	110	2015	10935	9928
32	-1336	256	2271	11943	10744
